In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon May 30 10:11:44 2022

@author: roy.369
"""


import numpy as np
import pickle

import matplotlib.pyplot as plt
import os
import sys
from scipy.interpolate import griddata
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colorbar
from matplotlib import rc
mpl.rcParams['axes.linewidth'] = 3
import matplotlib as mpl  
mpl.rc('font',family='Times New Roman')
from matplotlib.pyplot import cm
from numpy.polynomial.polynomial import polyfit
from scipy.optimize import curve_fit
from matplotlib.ticker import ScalarFormatter

In [28]:
def arr_loc(val,X):
   ind = 0
   for i in range(len(X)):
      if(round(X[i],2) == round(val,2)):
         ind = i
         break
   return ind


def power_fit(x,a1,m):
    return a1*x**m
  
def linear_fit(x,a,b):
    return a+b*x
    
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx

In [29]:
def r_point_upper_half_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_1 = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              r_lab.append(r_pair_1)  
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_1 = r_pair_1+1

    R_relative_1 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_1,r_pair_1

In [30]:
def r_point_full_grid(N):

    r_lab = []
    r_rel_x = []
    r_rel_y = []
    x_span = int((int(N))/2)+1
    r_pair_2 = 0
    for i in range(x_span):
        for j in range(x_span):
              r_lab.append(r_pair_2)
              r_rel_x.append(i)
              r_rel_y.append(j)
              r_pair_2 = r_pair_2+1

    R_relative_2 = np.stack((r_rel_x,r_rel_y),axis = 1)
    return R_relative_2,r_pair_2

In [31]:
def k_point_grid_upper_half_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
            if(i<=j):
              K_lab.append(k_pair)  
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


def k_point_grid_full_bz(N):

    K_lab = []
    Kx = []
    Ky = []
    x_span = int((int(N))/2)+1
    k_pair = 0
    for i in range(x_span):
        for j in range(x_span):
              K_lab.append(k_pair)
              Kx.append(i)
              Ky.append(j)
              k_pair = k_pair+1
    BZ = np.stack((K_lab,Kx,Ky),axis = 1)
    
    return BZ,k_pair


In [32]:
def adj_mat_calc(N):

    r_grid, r_rel_grid_size = r_point_upper_half_grid(int(N))
    r_rel_x = r_grid[:,0]
    r_rel_y = r_grid[:,1]

    r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
    r_2_unq = np.sort(np.unique(r_2))

    rx_cord = []
    ry_cord = []
    r2 = []
    rx = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    ry = np.arange(int(int(N)/2),-1*int(int(N)/2),-1)
    for i in range(len(rx)):
        for j in range(len(ry)):
            rx_cord.append(rx[i])
            ry_cord.append(ry[j])
            r2.append(rx[i]*rx[i]+ry[j]*ry[j])

    #plt.figure()
    #plt.scatter(rx_cord,ry_cord,c=r2)

    adj_mat = np.zeros(len(r_2_unq))
    for i in range(len(r_2_unq)):
        count = 0
        for i2 in range(len(r2)):
            if round(r2[i2],2) == round(r_2_unq[i],2):
               count = count+1
        adj_mat[i] = count
    
    return np.stack((r_2_unq,np.asarray(adj_mat)),axis = 1)  

In [33]:
def plot_func_multiple_beta_hf(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(["red","orange","goldenrod","blue","purple"]) #(cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (25,20))
   #plt.title(plot_title,fontsize = 60)
   plt.xticks(fontsize = 80)
   plt.yticks(fontsize = 80)
   #plt.xscale('log')
   #plt.yscale('log') 
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   for jj in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[jj]))
       T = round(1/beta,2)
       plt.errorbar(x_val,y_val[:,jj],yerr = y_val_err[:,jj],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))
   plt.grid(True,which='both')

   plt.axvspan(3.1, 4.3, facecolor='r', alpha=0.2)
   plt.legend(loc = 'best',nrow = 2,fontsize = 60)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()

In [34]:
def plot_func_multiple_beta_doping(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(["red","orange","goldenrod","blue","purple"]) #(cm.gnuplot(np.linspace(0, 1, len(trot)+1)))
   plt.figure(figsize = (28,20))
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 80)
   plt.yticks(fontsize = 80)
   for k in range(len(trot)):
       c_1 = next(color_1)
       beta = (float(dtau))*(float(trot[k]))
       T = round(1/beta,2)
       plt.errorbar(x_val[:,k],y_val[:,k],yerr = y_val_err[:,k],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"T = %s"%str(T))
   plt.grid(True,which='both')
   plt.legend(loc = 'best',ncol = int(len(trot)/2),fontsize = 50)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()

In [47]:
def plot_func_multiple_U_doping(N,u_val,mu,dtau,trot,plot_title,plot_x_label,plot_y_label,x_val,y_val,y_val_err,plot_name):

   color_1 = iter(['blue','darkgreen','olive','darkgoldenrod','orange','red'])   #cm.tab20(np.linspace(0, 1, len(u_val)+1)))
   plt.figure(figsize = (28,20))
   #plt.title(plot_title,fontsize = 60)
   plt.xticks([0.0,0.5,1.0,1.5,2.0],fontsize = 80)
   plt.yticks(fontsize = 80)
   #plt.xscale('log')
   #plt.yscale('log') 
   #plt.ylabel(plot_y_label,fontsize = 60)
   #plt.xlabel(plot_x_label,fontsize = 60)
   #plt.errorbar(x_val_met,y_val_met,yerr = y_val_err_met,c='blue',marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"U = 1.0")
   for i in range(len(u_val)):
       c_1 = next(color_1)
       plt.errorbar(x_val[i,:],y_val[i,:],yerr = y_val_err[i,:],c=c_1,marker = 'o',markersize = 20, elinewidth = 3, capsize = 5,label = r"U = %s"%str(u_val[i]))
   plt.grid(True,which='both')
   plt.legend(loc = 'best',fontsize = 50)
   plt.tight_layout()
   plt.savefig(plot_name)
   plt.close()

In [48]:
def equal_time_density_density_correlation_function_half_filling(Text_dir_main_corr_hf,Text_dir_main_eqm,Graph_dir_hf,N,u,mu,dtau,L):

   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Den_Den_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Den_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))
   
   Den_Den_conn_corr_hf = np.zeros((len(u),len(L),len(r_2_unq)))
   Den_Den_conn_corr_hf_std = np.zeros((len(u),len(L),len(r_2_unq)))

    
   Nden = np.zeros((len(u),len(L)))
   Nden_std = np.zeros((len(u),len(L)))
    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       
       
   for i in range(len(u)):
       for k in range(len(L)):
                       
           Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,mu,dtau,L[k])

           filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_avg, 'rb') as infile:
                sys_measure_avg = pickle.load(infile)

           filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],mu,dtau,L[k])
           with open(filename_eqm_std, 'rb') as infile:
                sys_measure_std = pickle.load(infile)

           Nden[i][k] = sys_measure_avg['Density averaged']
           Nden_std[i][k] = sys_measure_std['Density standard deviation']
           
           Text_dir_den_den_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])
           Text_dir_den_den_conn_corr_hf = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr_hf,N,u[i],dtau,mu,dtau,L[k])

           filename_den_den_corr = '%s/Equal_time_Density_Density_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Den_corr_hf[i,k,:],Den_Den_corr_hf_std[i,k,:] = np.loadtxt(filename_den_den_corr,unpack = 'True', usecols = [1,2])

           filename_den_den_conn_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_conn_corr_hf,N,u[i],mu,dtau,L[k])
           Den_Den_conn_corr_hf[i,k,:],Den_Den_conn_corr_hf_std[i,k,:] = np.loadtxt(filename_den_den_conn_corr,unpack = 'True', usecols = [1,2])


 
   Den_Den_corr_hf_n0 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n1 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n2 = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n3 = np.zeros((len(u),len(L)))

   Den_Den_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Den_Den_corr_hf_n3_std = np.zeros((len(u),len(L)))

   Den_Den_conn_corr_hf_n0 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n1 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n2 = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n3 = np.zeros((len(u),len(L)))
    
   Den_Den_conn_corr_hf_n0_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n1_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n2_std = np.zeros((len(u),len(L)))
   Den_Den_conn_corr_hf_n3_std = np.zeros((len(u),len(L)))


   for i in range(len(u)):
       for k in range(len(L)):

           Den_Den_corr_hf_n0[i][k] = Den_Den_corr_hf[i][k][0]
           Den_Den_corr_hf_n1[i][k] = Den_Den_corr_hf[i][k][1]
           Den_Den_corr_hf_n2[i][k] = Den_Den_corr_hf[i][k][2]
           Den_Den_corr_hf_n3[i][k] = Den_Den_corr_hf[i][k][3]

           Den_Den_corr_hf_n0_std[i][k] = Den_Den_corr_hf_std[i][k][0]
           Den_Den_corr_hf_n1_std[i][k] = Den_Den_corr_hf_std[i][k][1]
           Den_Den_corr_hf_n2_std[i][k] = Den_Den_corr_hf_std[i][k][2]
           Den_Den_corr_hf_n3_std[i][k] = Den_Den_corr_hf_std[i][k][3]

           Den_Den_conn_corr_hf_n0[i][k] = Den_Den_conn_corr_hf[i][k][0]
           Den_Den_conn_corr_hf_n1[i][k] = Den_Den_conn_corr_hf[i][k][1]
           Den_Den_conn_corr_hf_n2[i][k] = Den_Den_conn_corr_hf[i][k][2]
           Den_Den_conn_corr_hf_n3[i][k] = Den_Den_conn_corr_hf[i][k][3]
           
           Den_Den_conn_corr_hf_n0_std[i][k] = Den_Den_conn_corr_hf_std[i][k][0]
           Den_Den_conn_corr_hf_n1_std[i][k] = Den_Den_conn_corr_hf_std[i][k][1]
           Den_Den_conn_corr_hf_n2_std[i][k] = Den_Den_conn_corr_hf_std[i][k][2]
           Den_Den_conn_corr_hf_n3_std[i][k] = Den_Den_conn_corr_hf_std[i][k][3]
           
           
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"

    
   Pl_name_n1 = "%s/Density_density_correlation_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)
   Pl_name_conn_n1 = "%s/Density_density_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_half_filling_N_%s_dtau_%s.png"%(Graph_dir_hf,N,dtau)

  
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n1,Pl_x_lab,Pl_y_lab,U_val,Den_Den_corr_hf_n1,Den_Den_corr_hf_n1_std,Pl_name_n1)
   plot_func_multiple_beta_hf(N,u,mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,U_val,Den_Den_conn_corr_hf_n1,Den_Den_conn_corr_hf_n1_std,Pl_name_conn_n1)
    



In [49]:
def equal_time_moment_holon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,u,Mu,dtau,L):


   Adj_list = adj_mat_calc(N)
    
   r_grid, r_rel_grid_size = r_point_upper_half_grid(N)
   r_rel_x = r_grid[:,0]
   r_rel_y = r_grid[:,1]

   r_2 = np.add(np.power(r_rel_x,2),np.power(r_rel_y,2))
   r_2_unq = np.sort(np.unique(r_2))

   Den_Den_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Den_Den_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   
   Den_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Den_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))

   Dbn_Dbn_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Dbn_Dbn_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
    
   Mmt_Hln_conn_corr = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))
   Mmt_Hln_conn_corr_std = np.zeros((len(u),len(Mu),len(L),len(r_2_unq)))   
   
   Nden = np.zeros((len(u),len(Mu),len(L)))
   Nden_std = np.zeros((len(u),len(Mu),len(L)))

       
   for i in range(len(u)):
       for j in range(len(Mu)):
           for k in range(len(L)):
                
               Text_dir_eqm = "%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Thermodynamic_measurements"%(Text_dir_main_eqm,N,u[i],dtau,Mu[j],dtau,L[k])

               filename_eqm_avg = '%s/Thermodynamic_measurements_normal_averaged_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],Mu[j],dtau,L[k])
               with open(filename_eqm_avg, 'rb') as infile:
                    sys_measure_avg = pickle.load(infile)

               filename_eqm_std = '%s/Thermodynamic_measurements_normal_standard_deviation_dictionary_N_%s_U_%s_mu_%s_dtau_%s_L_%s.pkl' %(Text_dir_eqm,N,u[i],Mu[j],dtau,L[k])
               with open(filename_eqm_std, 'rb') as infile:
                    sys_measure_std = pickle.load(infile)

               Nden[i][j][k] = sys_measure_avg['Density averaged']
               Nden_std[i][j][k] = sys_measure_std['Density standard deviation']  
               
               Text_dir_den_den_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_density_connected_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
           
               filename_den_den_corr = '%s/Equal_time_Density_Density_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_den_corr,N,u[i],Mu[j],dtau,L[k])
               Den_Den_conn_corr[i,j,k,:],Den_Den_conn_corr_std[i,j,k,:] = np.loadtxt(filename_den_den_corr,unpack = 'True', usecols = [1,2])

               Text_dir_dbn_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Doublon_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
           
               filename_dbn_dbn_corr = '%s/Equal_time_Doublon_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_dbn_dbn_corr,N,u[i],Mu[j],dtau,L[k])
               Dbn_Dbn_conn_corr[i,j,k,:],Dbn_Dbn_conn_corr_std[i,j,k,:] = np.loadtxt(filename_dbn_dbn_corr,unpack = 'True', usecols = [1,2])

               Text_dir_den_dbn_corr = '%s/Text_files_N_%s_U_%s_dtau_%s/Mu_%s/dtau_%s_L_%s/Density_doublon_connected_correlation_functions'%(Text_dir_main_corr,N,u[i],dtau,Mu[j],dtau,L[k])
           
               filename_den_dbn_corr = '%s/Equal_time_Density_Doublon_connected_correlation_real_space_neighbor_averaged_normal_avg_N_%s_U_%s_mu_%s_dtau_%s_L_%s.dat'%(Text_dir_den_dbn_corr,N,u[i],Mu[j],dtau,L[k])
               Den_Dbn_conn_corr[i,j,k,:],Den_Dbn_conn_corr_std[i,j,k,:] = np.loadtxt(filename_den_dbn_corr,unpack = 'True', usecols = [1,2])


               Mmt_Hln_conn_corr[i,j,k,:] = np.subtract(3*Den_Dbn_conn_corr[i,j,k,:],np.add(Den_Den_conn_corr[i,j,k,:],2*Dbn_Dbn_conn_corr[i,j,k,:]))
               Mmt_Hln_conn_corr_std[i,j,k,:] = np.sqrt(np.add(9*np.power(Den_Dbn_conn_corr_std[i,j,k,:],2),np.add(np.power(Den_Den_conn_corr_std[i,j,k,:],2),4*np.power(Dbn_Dbn_conn_corr_std[i,j,k,:],2))))

    
   U_val = np.zeros(len(u))
   for i in range(len(u)):
       U_val[i] = float(u[i])
       

   Mu_val = np.zeros(len(Mu))
    
   for j in range(len(Mu)):
       Mu_val[j] = float(Mu[j])


   Mmt_Hln_conn_corr_n0 = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n1 = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n2 = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n3 = np.zeros((len(u),len(Mu),len(L)))
    
   Mmt_Hln_conn_corr_n0_std = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n1_std = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n2_std = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_n3_std = np.zeros((len(u),len(Mu),len(L)))

   Mmt_Hln_conn_corr_tot = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_nl = np.zeros((len(u),len(Mu),len(L))) 
   Mmt_Hln_conn_corr_tot_std = np.zeros((len(u),len(Mu),len(L)))
   Mmt_Hln_conn_corr_nl_std = np.zeros((len(u),len(Mu),len(L))) 

    
   for i in range(len(u)):
       for j in range(len(Mu)):
           for k in range(len(L)):

               Mmt_Hln_conn_corr_n0[i][j][k] = Mmt_Hln_conn_corr[i][j][k][0]
               Mmt_Hln_conn_corr_n1[i][j][k] = Mmt_Hln_conn_corr[i][j][k][1]
               Mmt_Hln_conn_corr_n2[i][j][k] = Mmt_Hln_conn_corr[i][j][k][2]
               Mmt_Hln_conn_corr_n3[i][j][k] = Mmt_Hln_conn_corr[i][j][k][3]
           
               Mmt_Hln_conn_corr_n0_std[i][j][k] = Mmt_Hln_conn_corr_std[i][j][k][0]
               Mmt_Hln_conn_corr_n1_std[i][j][k] = Mmt_Hln_conn_corr_std[i][j][k][1]
               Mmt_Hln_conn_corr_n2_std[i][j][k] = Mmt_Hln_conn_corr_std[i][j][k][2]
               Mmt_Hln_conn_corr_n3_std[i][j][k] = Mmt_Hln_conn_corr_std[i][j][k][3]

           
 
   Pl_title_n0 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=0.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n1 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.0, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n2 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=1.414, $N=%sx%s, n = 1$"%(N,N)
   Pl_title_n3 = r"$\langle h_i d_j \rangle_{C}$ |i-j|=2.0, $N=%sx%s, n = 1$"%(N,N)

   Pl_y_lab = r"$\langle h_i d_j \rangle_{C}$"
   Pl_x_lab = r"$U$"


   for i in range(len(u)):
       
       Pl_name_conn_n1 = "%s/Moment_holon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
       Pl_name_conn_n0 = "%s/Moment_holon_correlation_connected_onsite_normal_averaged_N_%s_U_%s_dtau_%s.png"%(Graph_dir,N,u[i],dtau)
                     
  
       plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Mmt_Hln_conn_corr_n1[i,:,:],Mmt_Hln_conn_corr_n1_std[i,:,:],Pl_name_conn_n1)
       plot_func_multiple_beta_doping(N,u[i],Mu,dtau,L,Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[i,:,:],Mmt_Hln_conn_corr_n0[i,:,:],Mmt_Hln_conn_corr_n0_std[i,:,:],Pl_name_conn_n0)

    
   for k in range(len(L)):
       
       Pl_name_conn_n1 = "%s/Moment_holon_correlation_connected_1st_nearest_neighbor_averaged_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       Pl_name_conn_n0 = "%s/Moment_holon_correlation_connected_onsite_normal_averaged_N_%s_dtau_%s_L_%s.png"%(Graph_dir,N,dtau,L[k])
       
       plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n2,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Mmt_Hln_conn_corr_n1[:,:,k],Mmt_Hln_conn_corr_n1_std[:,:,k],Pl_name_conn_n1)
       plot_func_multiple_U_doping(N,u,Mu,dtau,L[k],Pl_title_n0,Pl_x_lab,Pl_y_lab,Nden[:,:,k],Mmt_Hln_conn_corr_n0[:,:,k],Mmt_Hln_conn_corr_n0_std[:,:,k],Pl_name_conn_n0)
             


In [50]:
def main():

    N = "10"
    Dtau = "0.05"

    U=["1.0","4.0","5.0","6.0","8.0","10.0"] #,"4.50","4.60","4.70"]
    Mu = ["-10.00","-9.50","-9.00","-8.50","-8.00","-7.50","-7.00","-6.50","-6.00","-5.50","-5.00","-4.50","-4.00","-3.50","-3.00","-2.50",
          "-2.00","-1.50","-1.00","-0.50","0.00","0.50","1.00","1.50","2.00","2.50","3.00","3.50","4.00","4.50","5.00","5.50","6.00","6.50","7.00",
          "7.50","8.00","8.50","9.00","9.50","10.00"]
    Trot = ["30","40"] #,"30","40","50","60"]

    
    Text_dir_main_corr = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s_real_space_correlations'%N
    Text_dir_main_eqm = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Text_files/Text_files_N_%s'%N


    Graph_dir = '/Users/roy.369/Documents/Cold_atom_correlators/FHM_Legacy_data/FHM_Roy_data/Graphs/Graphs_N_%s/Graphs_N_%s_dtau_%s_real_space_correlations_normal_averaged/Moment_holon_correlations'%(N,N,Dtau)
    if not os.path.exists(Graph_dir):
        os.makedirs(Graph_dir)
        
    equal_time_moment_holon_correlation_function_doping(Text_dir_main_corr,Text_dir_main_eqm,Graph_dir,N,U,Mu,Dtau,Trot)





In [51]:
main()